<a href="https://colab.research.google.com/github/engkimo/ML-Experimental/blob/main/mPLUG_DocOwl2_%E6%A4%9C%E8%A8%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# git-lfs の初期化
!git lfs install
# モデルのクローン
!git clone https://huggingface.co/mPLUG/DocOwl2

Git LFS initialized.
Cloning into 'DocOwl2'...
remote: Enumerating objects: 43, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (39/39), done.
remote: Total 43 (delta 9), reused 0 (delta 0), pack-reused 4 (from 1)
Unpacking objects: 100% (43/43), 407.97 KiB | 11.03 MiB/s, done.
Encountered 1 file(s) that may not have been copied correctly on Windows:
	model.safetensors

See: `git lfs help smudge` for more details.


In [4]:
!nvidia-smi
!pip cache purge
!pip uninstall -y bitsandbytes
!pip uninstall -y transformers
!pip uninstall -y torch torchvision

!pip install flash_attn
!pip install -q torch torchvision --index-url https://download.pytorch.org/whl/cu118
!pip install -q transformers
!pip install -q bitsandbytes
!pip install -q transformers accelerate
!pip install -q icecream


Wed Nov 20 02:49:06 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0              49W / 400W |      2MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [10]:
!pip install flash_attn

  Using cached flash_attn-2.7.0.post2.tar.gz (2.7 MB)
  Preparing metadata (setup.py) ... done
  Created wheel for flash_attn: filename=flash_attn-2.7.0.post2-cp310-cp310-linux_x86_64.whl size=189287885 sha256=7ccce59f987f422d8210587383914057cf7db219988427ef71287e89153ad2fa
  Stored in directory: /root/.cache/pip/wheels/bf/e3/ed/5e845387d52f2debd1bafb847bf3d774d3f0a3c8e31b1dc948
Successfully built flash_attn


In [6]:
import torch
from transformers import AutoTokenizer, AutoModel, BitsAndBytesConfig
import gc
import os
from icecream import ic

In [7]:
class DocOwlInfer():
    def __init__(self, ckpt_path):
        self.tokenizer = AutoTokenizer.from_pretrained(ckpt_path, use_fast=False)
        self.model = AutoModel.from_pretrained(
            ckpt_path,
            trust_remote_code=True,
            low_cpu_mem_usage=True,
            torch_dtype=torch.float16,
            device_map='auto'
        )
        self.model.init_processor(
            tokenizer=self.tokenizer,
            basic_image_size=504,
            crop_anchors='grid_12'
        )

    def inference(self, images, query):
        messages = [{'role': 'USER', 'content': '<|image|>'*len(images)+query}]
        answer = self.model.chat(messages=messages, images=images, tokenizer=self.tokenizer)
        return answer

In [ ]:
# class DocOwlInfer():
#     def __init__(self, ckpt_path):
#         # メモリクリア
#         torch.cuda.empty_cache()
#         gc.collect()

#         # トークナイザーの初期化
#         self.tokenizer = AutoTokenizer.from_pretrained(
#             ckpt_path,
#             use_fast=False
#         )

#         # モデルの初期化（8bit量子化なし）
#         self.model = AutoModel.from_pretrained(
#             ckpt_path,
#             trust_remote_code=True,
#             low_cpu_mem_usage=True,
#             torch_dtype=torch.float16,
#             device_map="auto",
#             max_memory={
#                 0: "12GB",  # GPUメモリ制限
#                 "cpu": "32GB"
#             }
#         )

#         # プロセッサの初期化（より小さいサイズに）
#         self.model.init_processor(
#             tokenizer=self.tokenizer,
#             basic_image_size=160,  # さらに小さく
#             crop_anchors='grid_4'
#         )

#     @torch.inference_mode()
#     def inference(self, images, query, batch_size=1):
#         try:
#             # 1枚ずつ処理
#             results = []
#             for i in range(0, min(len(images), batch_size)):
#                 messages = [{'role': 'USER', 'content': '<|image|>'+query}]

#                 # メモリクリア
#                 torch.cuda.empty_cache()
#                 gc.collect()

#                 # 1枚の画像で推論
#                 answer = self.model.chat(
#                     messages=messages,
#                     images=[images[i]],
#                     tokenizer=self.tokenizer
#                 )
#                 results.append(answer)

#             return " ".join(results)

#         except Exception as e:
#             return f"推論中にエラーが発生しました: {str(e)}"

# def initialize_model_safely():
#     try:
#         print("\nGPU情報:")
#         !nvidia-smi

#         print("\nPyTorchのバージョンとCUDAの確認:")
#         print(f"PyTorch: {torch.__version__}")
#         print(f"CUDA available: {torch.cuda.is_available()}")
#         if torch.cuda.is_available():
#             print(f"CUDA device: {torch.cuda.get_device_name(0)}")
#             print(f"Current GPU memory usage: {torch.cuda.memory_allocated()/1024**3:.2f} GB")

#         print("\nモデルの初期化を開始...")
#         docowl = DocOwlInfer(ckpt_path='mPLUG/DocOwl2')
#         print("モデルの初期化が完了")
#         return docowl

#     except Exception as e:
#         print(f"初期化中にエラーが発生しました: {str(e)}")
#         print("\n詳細なエラー情報:")
#         import traceback
#         traceback.print_exc()
#         return None

In [11]:
  # モデルの初期化
  docowl = DocOwlInfer(ckpt_path='mPLUG/DocOwl2')

visual_encoder.py:   0%|          | 0.00/20.4k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/mPLUG/DocOwl2:
- visual_encoder.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/mPLUG/DocOwl2:
- visual_compressor.py
- visual_encoder.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


model.safetensors:   0%|          | 0.00/17.1G [00:00<?, ?B/s]

Some weights of MPLUGDocOwl2 were not initialized from the model checkpoint at mPLUG/DocOwl2 and are newly initialized: ['model.layers.0.self_attn.rotary_emb.inv_freq', 'model.layers.1.self_attn.rotary_emb.inv_freq', 'model.layers.10.self_attn.rotary_emb.inv_freq', 'model.layers.11.self_attn.rotary_emb.inv_freq', 'model.layers.12.self_attn.rotary_emb.inv_freq', 'model.layers.13.self_attn.rotary_emb.inv_freq', 'model.layers.14.self_attn.rotary_emb.inv_freq', 'model.layers.15.self_attn.rotary_emb.inv_freq', 'model.layers.16.self_attn.rotary_emb.inv_freq', 'model.layers.17.self_attn.rotary_emb.inv_freq', 'model.layers.18.self_attn.rotary_emb.inv_freq', 'model.layers.19.self_attn.rotary_emb.inv_freq', 'model.layers.2.self_attn.rotary_emb.inv_freq', 'model.layers.20.self_attn.rotary_emb.inv_freq', 'model.layers.21.self_attn.rotary_emb.inv_freq', 'model.layers.22.self_attn.rotary_emb.inv_freq', 'model.layers.23.self_attn.rotary_emb.inv_freq', 'model.layers.24.self_attn.rotary_emb.inv_freq', 

generation_config.json:   0%|          | 0.00/162 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuratio

In [18]:
# まず1枚だけテスト
images = ['./DocOwl2/examples/AZ3c55a48d9c.01_stdR3.jpg']
if docowl is not None:
    print("\n推論を開始します...")
    docowl.inference(images, query='This document is the billing for gas rates in Japan. Please list the items that can be obtained.')
else:
    print("\nモデルの初期化に失敗しました。上記のエラー情報を確認してください。")


推論を開始します...


ic| prompt: ('USER: <img 1><|image|>This document is the billing for electricity rates in '
             'Japan.Please list the items that can be obtained. ASSISTANT:')


 2


In [26]:
docowl.inference(images, query='この文書は東京ガスの請求書です。構造を捉え,key-valueの出力をしてください。')

ic| prompt: 'USER: <img 1><|image|>この文書は東京ガスの請求書です。構造を捉え,key-valueの出力をしてください。 ASSISTANT:'
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


{'title': 'TOKYO GAS', 'page': 1, 'sections': [{'title': '受付番号', 'page': '1', 'Text': '0570-002299 03-3912-1850\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n


"{'title': 'TOKYO GAS', 'page': 1, 'sections': [{'title': '受付番号', 'page': '1', 'Text': '0570-002299 03-3912-1850\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n"